# DS4DS Homework Exercise Sheet 05

### General Rules
- Only write within the area indicated by `#--- YOUR CODE [STARTS|ENDS] HERE ---#`
- Do not import any additional libraries without our explicit instructions.
- Do not delete or insert cells (If you are using a Jupyter IDE, you are prevented from doing so).

### Task 1: Regularization - (3.5 points)

In the first task, the same data as already used in Ex04 task3 will be utilized.
The given data was produced using a polynomial of the form


$$
\begin{align}
    y = w_0 + x \cdot w_1 + \dots + x^p \cdot w_p
\end{align}
$$

with an unknown degree $p$.

Below you are given data for the inputs $x$ and the outputs $y$.  All of the data for the measurements $y$ is affected by bias-free, additive, Gaussian noise, i.e., the values you are given below do not fully follow the formula given above, but instead :

$$
\begin{align}
    y = w_0 + x \cdot w_1 + \dots + x^p \cdot w_p + \nu \quad \mathrm{with} \, \nu \sim \mathcal{N}(0, \sigma_n^2).
\end{align}
$$

The variance of the noise process is unknown, but it is also not necessary to estimate it for completing this task.


The task in the following is to identify the coefficents using the training data and validate the result using the validation data, like already done in Ex04. 
But in this task we will assume a fixed degree of the polynom $p = 6$ for our model. 

**Model:** We will make use of:

$$
\begin{align}
    y = w_0 + x \cdot w_1 + x^2 \cdot w_2 + x^3 \cdot w_3 + x^4 \cdot w_4 + x^5 \cdot w_5 + x^6 \cdot w_6 .
\end{align}
$$


In [1]:
using MAT
using Statistics
using LinearAlgebra
using Random

In [2]:
data_1 = matopen("data_task_3_ex04.mat");

# training data
x_training = read(data_1, "x_training")[1, :];
y_training = read(data_1, "y_training")[1, :];

# validation data
x_validation = read(data_1, "x_validation")[1, :];
y_validation = read(data_1, "y_validation")[1, :];


**a)** Based on the implementation of EX04, calculate the estimation error of an OLS estimator, which uses the model defined above. **- (0.5 points)**

**Hint**: The functions from Ex04 can be used here. 

In [3]:
function regressor_matrix(x, p)
	"""
	Args:
		x: Input data
		p: Degree of the polynomial

	Returns:
		Regressor matrix Z with shape (length(x), p+1)
	"""
	@assert ndims(x) == 1
	#--- YOUR CODE STARTS HERE ---#
	Z = zeros(length(x), p+1)
    for i = 0:p
        Z[:,i+1] = x.^i
    end
	#--- YOUR CODE ENDS HERE ---#

	return Z
end


regressor_matrix (generic function with 1 method)

In [4]:
function compute_parameter_estimation(x, y, p)
	"""
	Args:
		x: Input data
		y: Output data
		p: Degree of the polynomial

	Returns:
		parameter vector w with shape (p+1)
	"""
	@assert ndims(x) == ndims(y) == 1
	#--- YOUR CODE STARTS HERE ---#
	Z = regressor_matrix(x, p)
	w = inv(Z' * Z) * Z' * y
	#--- YOUR CODE ENDS HERE ---#

	return w
end


compute_parameter_estimation (generic function with 1 method)

In [5]:
function error_function(a, b)
	"""Computes the error between the two input vectors."""

	@assert length(a) == length(b)
	@assert ndims(a) == ndims(b) == 1

	N = length(a)
	return sqrt(1 / N * sum((a - b) .^ 2))
end


error_function (generic function with 1 method)

In [6]:
p = 6
w = compute_parameter_estimation(x_training, y_training, p)
y_est = regressor_matrix(x_validation, p) * w
err_OLS = error_function(y_est[:, 1], y_validation[:, 1])
println("Current error: ", err_OLS)

@assert ndims(err_OLS) == 0 "is err_OLS scalar?"
@assert 100 > err_OLS > 0



Current error: 38.23522411608347


In [7]:
# please leave this cell as it is


To reduce the error, the ridge regression should be applied in the following subtasks. 

**b)** For a proper scaling, the data should be normalized in the beginning. Therefore, write a function which normalizes the given data vector $d$ based on the following equation:

$$
\begin{align}
    d_{norm} = \frac{d - \mu(d)}{\sigma(d)}
\end{align}
$$

where $\mu(d)$ defines the mean of the data $d$ and $\sigma(d)$ defines the standard deviation, accordingly.

There are two common mistakes made when data is normalized:

1. $\mu$ and $\sigma$ are calculated on the union of training and test (or val) data. 
*Problem:* When you normalize the training data, information from the test (or val) data is introduced to the training data by the usage of $\mu$ and $\sigma$. This introduces bias to your tests (or vals) later on such that the results are overestimating the general performance.

2. Training and test (or val) data are normalized separately.
*Explanation:* Assume you have normalized the training data x_training, y_training with $\mu_{train}$ and $\sigma_{train}$ and trained an algorithm A on it. Now you want to know, how A would be doing if it is applied on the actual problem the data comes from. This is done by emulating the actual problem. In a real application of the trained A, you get some $\hat{x}$, normalize it with $\mu_{train}$ and $\sigma_{train}$, and apply A on it to get a prediction $\hat{y}_{pred}$ (within normalization space). This is mimiced in testing (or validation), i.e. you use $\mu_{train}$ and $\sigma_{train}$ to normalize your test (or val) data. Many people do a mistake here and calculate a separate $\mu_{test}$ and $\sigma_{test}$, which does not represent the real application any more and introduces bias. A sufficient way of thinking about normalization is, that it is part of your actual algorithm A with parameters $\mu_{train}$ and $\sigma_{train}$.

*Coming back to the assignment:* A common name for a normalizer is *StandardScaler*. There you store your $\mu_{train}$ and $\sigma_{train}$. The StandardScaler is used for two function. You see them below with further explanations. You are supposed to fill in the missing parts.**- (1 points)**

In [8]:
# Nothing to do in this cell!

# Dump for normalization parameters
mutable struct StandardScaler
    mean::Union{Float64, Nothing}
    std::Union{Float64, Nothing}
end

# Constructor
function StandardScaler()
    return StandardScaler(nothing, nothing)
end


StandardScaler

In [9]:
# Function for StandardScaler
function fit!(scaler::StandardScaler, X::Vector{Float64})
    # Fit the StandardScaler on the data
    #
    # Here you should calculate the mean and standard deviation on the given data vector X.
    scaler.mean = 0.  # TODO: Init this variable.
    scaler.std = 0.  # TODO: Init this variable.
    #--- YOUR CODE STARTS HERE ---#
    N = length(X)
    for i = 1:N
        scaler.mean += X[i]
    end
    scaler.mean /= N
    for i = 1:N
        scaler.std += (X[i] - scaler.mean)^2
    end
    scaler.std /= (N - 1)
    scaler.std = sqrt(scaler.std)
    #--- YOUR CODE ENDS HERE ---#
end

function transform(scaler::StandardScaler, X::Vector{Float64})
    # Transform the data using the fitted mean and std
    #
    # Here you should normalize the data X and return it.
    if scaler.mean === nothing || scaler.std === nothing
        throw(ArgumentError("The scaler has not been fitted. Please call fit! first."))
    end
    
    X_transformed = zeros(length(X))  # This shall be your transformed data which is returned.
    #--- YOUR CODE STARTS HERE ---#
    X_transformed = (X .- scaler.mean) ./ scaler.std
    #--- YOUR CODE ENDS HERE ---#
    return X_transformed
end


transform (generic function with 1 method)

In [10]:
test = convert(Vector{Float64}, collect(1:5))
SS = StandardScaler()  # Call to the constructor
fit!(SS, test)  # calculate mean and std on test for SS.
test_norm = transform(SS, test)
truth = [-1.2649110640673518, -0.6324555320336759, 0.0, 0.6324555320336759, 1.2649110640673518]

@assert size(test)[1] == size(test_norm)[1]
@assert transform(SS, test) == truth


In [11]:
# Do not change this cell!



In [12]:
# please leave this cell as it is


**c)** Implement the function below to calculate the parameters $\mathbf{w_{ridge}}$ using the ridge regression approach from the lecture.
The function should be able to generate a regressor matrix $\mathbf{Z}$ from the input vector $\mathbf{x}$ using the `regressor_matrix(x, p)` function implemented in subtask a) based on the model of a polynomial of degree $p$.
This regressor matrix $Z$ is then to be used together with the measurement vector $\mathbf{y}$ and the penatly term lambda to estimate the parameters $\mathbf{w_{ridge}}$ based on the ridge regression approach from the lecture.**- (1 point)**

In [13]:
function compute_parameter_estimation_ridge(x, y, p, lambda)
	"""
	Args:
		x: Input data
		y: Output data
		p: Degree of the polynomial
		lambda: scalar weigting factor


	Returns:
		parameter vector w_ridge with shape (p+1)
	"""
	@assert ndims(x) == ndims(y) == 1
	@assert ndims(p) == ndims(lambda) == 0
	#--- YOUR CODE STARTS HERE ---#
	Z = regressor_matrix(x, p)
	w_ridge = inv(Z' * Z + lambda * I) * (Z' * y)
	#--- YOUR CODE ENDS HERE ---#

	return w_ridge
end


compute_parameter_estimation_ridge (generic function with 1 method)

In [14]:
@assert isa(compute_parameter_estimation_ridge, Function)



In [15]:
# please leave this cell as it is


**d)** Use the StandardScaler from b) to normalize x_training and y_training and the implemented function in c) to evaluate the estimation error for different penalty factors lambda applying the ridge regression.

*Notes:*
- Consider only the given lambda values.
- Calculate the error with our error_function from above per lambda and store the minimum error among all identified models in the variable err_min.
- A polynomial of of degree $p = 6$ is to be used as the model.
- One StandardScaler for x_training and one for y_training.
- The error should be calculated in normalization space (i.e. compare y_pred to the normalized y_validation).
**- (1 point)**

In [16]:
p = 6 # degree of polynomial
lambda_vec = [0, 1/16, 1/8, 1/4, 1/2, 1, 2, 4, 8, 16] # lambdas to be evaluated

err_min = 10000.  # initialize minial error
SS_x = StandardScaler()  # StandardScaler for x-data.
SS_y = StandardScaler()  # StandardScaler for y-data.

#--- YOUR CODE STARTS HERE ---#
fit!(SS_x, x_training)
fit!(SS_y, y_training)
x_trafo = transform(SS_x, x_training)
y_trafo = transform(SS_y, y_training)
best_lambda = -1
Z = regressor_matrix(x_trafo, 6)
for l in lambda_vec
    w_ridge = compute_parameter_estimation_ridge(x_trafo, y_trafo, 6, l)
    preds = Z * w_ridge
    current_error = error_function(preds, y_trafo)
    if current_error < err_min
        err_min = current_error
        best_lambda = l
    end
end
#--- YOUR CODE ENDS HERE ---#

println("#######################")
println("Minimal error: ", err_min)


#######################
Minimal error: 0.26841819526957045


In [17]:
@assert isnan(err_min) == false "Your value for the minimal error is not a number."
@assert err_min < 5
@assert ndims(err_min) == 0 "is err_min scalar?"

### Task 2: Weighted least squares - (3.5 points)

#### Estimation of physical parameters of the barometric altitude formula:

The weighted least squares (WLS) approach is to be used to estimate the physical parameters $p_0$ and $\alpha$ of the barometric altitude formula

$$
\begin{align}
    p(h) = p_0 e^{\alpha \cdot h}.
\end{align}
$$

The parameter $p_0$ describes the air pressure at sea level and the parameter $\alpha$ is a constant that describes the exponential decrease in air pressure. 
Using the barometric altitude formula, the air pressure $p$ is defined as a function of the height $h$ above sea level.
To determine the parameters $p_0$ and $\alpha$, pressure measurements $p$ are available as a function of the height $h$. 
These were determined using 10 weather balloons with a pressure sensor and a line on which the balloon's ascent height can be varied.
The transmission of the pressure data at an altitude $h$ was transmitted to the ground station using an analog radio signal.
This radio signal is superimposed with a noise, whereby the variance of the noise increases with increasing altitude $h$ due to the greater
distance to the ground station.

<img src="weather-balloon.png" alt="measurements" width="400"/>

[(Picture source)](https://www.farmersalmanac.com/weather-balloons-19447)

The measured pressure values were recorded at altitudes of 150 m, 200 m, 250 m, ... , 2000 m (38 equidistant heights). 
The pressure measurements were realized in the unit $\text{Pa} = \frac{\text{N}}{\text{m}^2}$.

The results of the above-mentioned experiment are stored in the data set `data_task_2.mat`.
It contains a measurement matrix $p$ of the pressure and $h$ of the height. 
These measurement matrices have the dimension 38 × 10, whereby the rows each contain measured values of a constant height.



In [18]:
data_2 = matopen("data_task_2.mat");

p = read(data_2, "p");
h = read(data_2, "h");

p_vec = reduce(vcat, p);
h_vec = reduce(vcat, h);

**a)** The parameters occur non-linearly in the barometric altitude formula. Through which mathematical transformation can this non-linear estimation problem be transformed into a linear one such that linear LS (Least-Squares) can be applied? Write a function which implements this transformation to prepare the data for the use of linear LS methods.

The output of the function should be the regressor vector Z and the measurement vector Y, which can be used in linear LS methods **- (0.5 points)**

**Hint**: Use the provided, already concatenated vectors in this task.

In [19]:
function prepare_data(p_meas, h_meas)
	"""
	Function to linearize the pressure measurement of the form p_meas = p_0 exp(α h)

	Args:
		p_meas: Measured pressure data
		h_meas: Measured hight data
			
	Returns:
		Y: Measurement vector to be used in linear LS methods
		Z: Regressor matrix to be used in linear LS methods
	"""

	@assert ndims(p_meas) == ndims(h_meas) == 1

	#--- YOUR CODE STARTS HERE ---#
	# ln(p_meas) = ln(p_0) + α * h
	Y = log.(p_meas)
	Z = regressor_matrix(h_meas, 1)
	#--- YOUR CODE ENDS HERE ---#

	@assert ndims(Y) == 1
	@assert ndims(Z) == 2

	return Y, Z
end


prepare_data (generic function with 1 method)

In [20]:
@assert isa(prepare_data, Function)


x_test3 = [1.0, 1.0, 4.0, 2.0, 3.0, 1]
y_test3 = [2.1, 4.22, 20.66, 12.4, 14.23, 5.11]

Y_test, Z_test = prepare_data(x_test3, y_test3)



([0.0, 0.0, 1.3862943611198906, 0.6931471805599453, 1.0986122886681098, 0.0], [1.0 2.1; 1.0 4.22; … ; 1.0 14.23; 1.0 5.11])

In [21]:
# please leave this cell as it is


**b)** Write a function to calculate the parameters $p_0$ and $\alpha$ in the transformed space of the barometric height equation for the given data set. To do this, you can use the OLS-function from Ex04. **- (1 point)**

In [22]:
function parameter_calculation_OLS(Z, Y)
	"""
	Args:
		Z: regressor matrix with shape (n_measurements, n_regressors)
		Y: measurement_vector with shape (n_measurements)

	Returns:
		The parameter vector w with shape (n_parameters)
	"""

	#--- YOUR CODE STARTS HERE ---#
	w = inv(Z' * Z) * Z' * Y
	#--- YOUR CODE ENDS HERE ---#
	return w
end;


In [23]:
@assert isa(parameter_calculation_OLS, Function)
Y, Z = prepare_data(p_vec, h_vec)
w2_OLS = parameter_calculation_OLS(Z, Y)

@assert isnan(w2_OLS[1]) == false "Your value for a is NaN"
@assert isnan(w2_OLS[2]) == false "Your value for b is NaN"

@assert w2_OLS[1] > 0 "The first parameter must be positive, so that the air pressure above sea level is positive"
@assert w2_OLS[2] < 0 "The secound parameter must be negative, so that the air pressure decreases with increasing hight"




In [24]:
# please leave this cell as it is


Since the measurement's variance is increasing with the height, in the following the WLS should be applied to reduce the estimation error even further. 

**c)** 
To determine the weighting matrix for the WLS method, you need to know the co-variance matrix of the transformed measurement noise. Since we have 10 pressure measurements per height available, write a function, which calculates the variance of the pressure per height and uses that information to calulate to weighting matrix $W$ for the WLS as inverse of the co-variance matrix $R$, like presented in the lecture.  **- (1 point)**


In [25]:
function calculate_W(p_matrix)
	"""
	Args:
		p_matrix: measurement_matrix

	Returns:
		W: Weighting matrix as inverse of the co-variance matrix
		R: Co-variance matrix of the measurement matrix
	"""
	@assert ndims(p_matrix) == 2
	#--- YOUR CODE STARTS HERE ---#
	R = I(38) .* var(p, dims=2)
	W = inv(R)
	#--- YOUR CODE ENDS HERE ---#
	return W, R
end;


In [26]:
W, R = calculate_W(p)

@assert ndims(W) == ndims(R) == 2


In [27]:
# please leave this cell as it is


**d)** Use the calculated weigthing matrix to apply the WLS, like presented in the lecture, to estimate the parameters for $p_0$ and $\alpha$. 
Therefore, write a function (below) which takes the regressor matrix $Z$ and  measurement vector $y$ provided by the prepare_data-function from task a), as well as the weighting matrix $W$ calculated with the function implemented in task c) **- (1 point)**

In [28]:
function parameter_calculation_WLS(Z, Y, W)
	"""
	Args:
		Z: regressor matrix 
		Y: measurement_vector 
		W: weigthing matrix

	Returns:
		The parameter vector w
	"""

	#--- YOUR CODE STARTS HERE ---#
	w_wls = inv(Z' * (repeat(diag(W), 10) .* Z)) * Z' * (repeat(diag(W), 10) .* Y) 
	w_wls[1] = exp(w_wls[1])
	#--- YOUR CODE ENDS HERE ---#
	return w_wls
end;


In [29]:
@assert isa(parameter_calculation_OLS, Function)

Y, Z = prepare_data(p_vec, h_vec)
W, R = calculate_W(p)
w_WLS = parameter_calculation_WLS(Z, Y, W)

@assert isnan(w_WLS[1]) == false "Your value for a is NaN"
@assert isnan(w_WLS[2]) == false "Your value for b is NaN"

@assert w_WLS[1] > 0 "The first parameter must be positive, so that the air pressure above sea level is positive"
@assert w_WLS[2] < 0 "The secound parameter must be negative, so that the air pressure decreases with increasing hight"




### Task 3: Recursive least squares - (3 points)

The [Worldwide Harmonised Light Vehicles Test Procedure](https://en.wikipedia.org/wiki/Worldwide_Harmonised_Light_Vehicles_Test_Procedure) is used to determine, e.g., fuel consumption, CO2 emission, ... of traditional and hybrid cars.
The profile is shown in the figure below.

<img src="WLTP.png" alt="measurements" width="400"/>

[source: Skript Antriebe für umweltfreundliche Fahrzeuge, UPB](https://ei.uni-paderborn.de/lea/lehre/veranstaltungen/lehrangebote/antriebe-fuer-umweltfreundliche-fahrzeuge)

The shown velocity profile is used in an experiment on the street, while the power needed to drive a car fullfilling the WLTP profile is measured.

The task in the following is to identify uncertain and varying model parameters based on measurement data.
We will use the same model already intoduced in the lecture, to calculate the power required for a certain velocity:

$$
\begin{align}
    P[k] = m \cdot g \cdot C_\text{r} \cdot v[k] + \nu \cdot v[k]^2 + \frac{ρ A C_\text{d}}{2} \cdot v[k]^3 .
\end{align}
$$

The measured power based on the equation above can be seen in the following figure.

<img src="WLTP_power.png" alt="measurements" width="400"/>

The data "WLTP_rain.mat", which is provided, was obtained during a rainy day. As the tire-street contact is highly depending on the street's wetness level, the rolling resistance coefficient $C_\text{r}$ changed during the test drive due to varying rain conditions.
To track how that parameter changed during the measurement, the recursive least squares (RLS) approach should be applied in this task.
The other parameters can be assumed to be not affected by the rain and, therefore, stay constant.

In [30]:
data_WLTP_rain = matopen("WLTP_rain.mat");
v_WLTP = read(data_WLTP_rain, "v")[1, :];
P_WLTP = read(data_WLTP_rain, "P")[1, :];

m = 1500; # mass of car in kg
g = 9.81; # gravity in m/s^2
ρ = 1.29; # density of air in kg/m^3
A = 2.0; # frontal area of car in m^2
Cd = 0.35; # drag coefficient
Cr_dry_street = 0.015; # rolling resistance coefficient
η = 4.5; # Bearing friction coefficient in kg/s

a, b, c = m * g * Cr_dry_street, η, (ρ * A * Cd) / 2 # condense parameters


(220.725, 4.5, 0.45149999999999996)

**a)** Write a general function to calculate the parameters of a linear model using one step of the RLS shown in lecture.
The function should take the measurement and regressor value(s) and the covariance matrix and the parameter(s) of the last step k-1 as inputs and return the estimated parameter(s) and the new covariance matrix for the current stime step k. **- (1.5 points)**

In [31]:
function create_identity_matrix(n::Int)
	"""
	Args:
		n: length of the matrix

	Returns:
		I(n): Identity matrix or scalar
	"""
	if n == 1
		return 1
	else
		return I(n)
	end
end;


In [32]:
function parameter_calculation_RLS(Z, Y, λ, W_RLS_k_1, P_k_1)
	"""
	Args:
		Z: regressor column vector of size (1,n)
		Y: measurement value of size (1)
		λ: fogetting factor
		W_RLS_k_1: last parameter(s) of timestep k-1 of size (n)
		P_k_1: last covariance matrix of size (n,n)

	Returns:
		W_RLS: New parameter vector of size (n,1)
		P_k: New covariance matrix of size (n,n)
	"""
	@assert length(λ) == 1
	@assert length(Y) == 1

	if size(Z) == ()
		# test if regressor is a scalar
		@assert length(W_RLS_k_1) == length(P_k_1) == length(Z)
	else
		# test if regressor is a vector
		@assert size(Z)[1] == 1
		@assert size(Z)[2] == size(P_k_1)[1]
		@assert size(Z)[2] == size(P_k_1)[2]
		@assert size(W_RLS_k_1)[1] == size(Z)[2]
	end

	#--- YOUR CODE STARTS HERE ---#
	gain = P_k_1 * Z' / (λ .+ Z * P_k_1 * Z')
	W_RLS = W_RLS_k_1 .+ gain * (Y .- Z * W_RLS_k_1)
	P_k = (create_identity_matrix(length(W_RLS_k_1)) .- gain * Z) * P_k_1 / λ
	#--- YOUR CODE ENDS HERE ---#

	@assert length(W_RLS) == length(Z)

	return W_RLS, P_k
end;


In [33]:
# Code to check the implementation
# Model of form: y = a*x1 + b*x2, results in:
# Y_test = a*Z_test[1,:] + b*Z_test[2,:]
# w_rls_test = [a, b]

num_steps = 11
Z_test = ones(num_steps, 2)
Y_test = collect(1:num_steps)

λ_test = 0.5
w_rls_test = ones(2, num_steps)
w_rls_test[:, 1] = [0.1, 0.2]
P_test = 10 * I(2)

for k ∈ 2:num_steps
	w_rls_test[:, k], P_test = parameter_calculation_RLS(Z_test[k, :]', Y_test[k], λ_test, w_rls_test[:, k-1], P_test)
end

truth_1 = [0.1, 0.9292682926829268, 1.2735537190082646, 1.6587188612099648, 2.080033277870217, 2.528686543110395, 2.9964696548988505, 3.4768943121432723, 3.9653073228115505, 4.458593023824821, 4.95476894504044]
truth_2 = [0.2, 1.0292682926829269, 1.3735537190082647, 1.758718861209965, 2.1800332778702174, 2.6286865431103954, 3.096469654898851, 3.576894312143273, 4.065307322811551, 4.5585930238248205, 5.05476894504044]

@assert w_rls_test[1, :] == truth_1   # is parameter a in all steps k like expected?
@assert w_rls_test[2, :] == truth_2   # is parameter b in all steps k like expected?


In [34]:
# please leave this cell as it is


**b)**  Use the implemented RLS function to estimate the  parameter $C_\text{r}$ and store the results in the given vector Cr_RLS.


**Hint1**: According to car model stated above, the RLS output data has to be further processed to provide an estimate of the parameter $C_\text{r}$ and NOT the parameter $a =  m \cdot g \cdot C_\text{r}$.

**Hint2**: During the first half of the measurement profile, the street was comparatively dry (rather high $C_\text{r}$) and during the second half rather wet (rather low $C_\text{r}$). You can use this observation to check your estimated $C_\text{r}$ on a qualitative basis.

**- (1.5 points)**

In [35]:
w_0 = 210   # inital parameter of a = m*g*Cr
λ = 0.99    # forgetting factor

Cr_RLS = ones(length(v_WLTP) - 1)

num_steps = length(v_WLTP)
P = 10                      # Initialize the covariance
γ = zeros(num_steps)        # Initialize the gain
W_RLS = ones(num_steps - 1)
W_RLS[1] = w_0              # Initial value of the parameter

#--- YOUR CODE STARTS HERE ---#
Z = [v_WLTP v_WLTP.^2 v_WLTP.^3]
for i = 2:num_steps-1
    w_curr, p_curr = parameter_calculation_RLS(Z[1:i-1, :], P_WLTP[i-1], λ, W_RLS[1:i-1], P)
    W_RLS[i] = w_curr
    P = p_curr
end
Cr_RLS = W_RLS ./ (m * g)
#--- YOUR CODE ENDS HERE ---#


BoundsError: BoundsError: attempt to access Tuple{} at index [1]

In [36]:
@assert size(W_RLS)[1] == size(Cr_RLS)[1] == 1800
@assert zeros(length(Cr_RLS)) < Cr_RLS < 0.1 * ones(length(Cr_RLS))
@assert isapprox(W_RLS[1], 210.0, atol = 10e-3)



AssertionError: AssertionError: zeros(length(Cr_RLS)) < Cr_RLS < 0.1 * ones(length(Cr_RLS))